In [0]:
'''

!mkdir tb1
LOG_DIR = './tb1'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


'''

!mkdir tb1
LOG_DIR = './tb1'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

#! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#! unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



mkdir: cannot create directory ‘tb1’: File exists
http://3cb9604b.ngrok.io


In [0]:
import os
import numpy as np
np.shape(os.listdir('input/images'))

(56791,)

In [0]:
!ls

FlipkartGridStage2DataSetImages.zip  gdrive  input  sample_data


In [0]:
#!ls gdrive/'My Drive'/Flipkart/Mob-30/ 
#!cp gdrive/'My Drive'/Flipkart/Mob-30/model-0.87.h5 /content/
#!ls

In [0]:
!ls
#!rm -rf model-0.87.h5

FlipkartGridStage2DataSetImages.zip  input	    __pycache__  train.py
gdrive				     model-0.87.h5  sample_data


In [6]:
!rm -rf train.py
from google.colab import files
files.upload()
!ls

Saving train.py to train.py
adc.json			     sample_data
FlipkartGridStage2DataSetImages.zip  tb1
gdrive				     train.py
input				     weights
ngrok				     weights1
ngrok-stable-linux-amd64.zip	     weights_Res10to15.zip
Res-net12E.h5			     weights_Res5to10.zip


In [0]:
import os
import pandas as pd

import cv2
import numpy as np

from train import  IMAGE_SIZE, ALPHA, VALIDATION_DATASET_SIZE
from trainmobileNet import  IMAGE_SIZE, ALPHA, VALIDATION_DATASET_SIZE
DEBUG = True
import matplotlib.pyplot as plt


import matplotlib.patches as patches
#WEIGHTS_FILE = "model_alfa025_size128-0.77.h5"
#WEIGHTS_FILE = "model_alfa05_size160-0.81.h5"
WEIGHTS_FILE = "weights/Res--610.53--0.83.h5"
#WEIGHTS_FILE = "Colab Outputs/Mobilenet 10 epochs/model-0.82.h5"
from keras.models import load_model
# Assuming your model includes instance of an "AttentionLayer" class
lab=pd.read_csv('input/test.csv')
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import *
from keras.utils import Sequence
from keras.callbacks import TensorBoard
from keras import layers
from keras.activations import relu

In [0]:
from keras.applications.resnet50 import ResNet50
def create_model(size, alpha):
    
    model_net = ResNet50(input_shape=(size[1], size[0], 3), include_top=False, weights = None)
    #model_net =Xception(input_shape=(size[1], size[0], 3), include_top=False, weights = None)
   # for i in range(len(model_net.layers) - 13):
    #    model_net.layers[i].trainable=False
    #x = _depthwise_conv_block(model_net.layers[-1].output, 1024/4, alpha, 1, block_id=14)
    #x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(4, kernel_size=(1, 1), padding="same")(model_net.layers[-1].output)
    x = GlobalAveragePooling2D()(x)
    #x = Reshape((4, ))(x)
    Model1=Model(inputs=model_net.input, outputs=x)
    
    print(Model1.summary())
    #

    return Model(inputs=model_net.input, outputs=x)

In [0]:
xmin=[]
xmax=[]
ymin=[]
ymax=[]
new_df=pd.DataFrame()

In [21]:
from tqdm import tqdm
def main():
    model = create_model([640,480], ALPHA)
    model.load_weights(WEIGHTS_FILE)
    #model = create_model(IMAGE_SIZE, ALPHA)
    #model.load_weights(WEIGHTS_FILE)

    ious = []
 
    
   # lbl2 = pd.read_csv('custom_labeled.csv')
   # lbl=pd.concat([lbl1, lbl2[1:]], ignore_index=True
   # labels=lbl[1000:]

    for image_name in tqdm(lab['image_name'].values):

        image_path=os.path.join('input/images/', image_name)
        image_ = cv2.imread(image_path)
        
        image = cv2.resize(image_, (640, 480))
        region = model.predict(np.array([image_]))
        #
        xmin.append(region[0][0])
        ymin.append(region[0][1])
        xmax.append(region[0][2])
        ymax.append(region[0][3])
        #print(xmin,ymin,xmax,ymax)
        #cv2.rectangle(image,(xmin,ymin),(xmax,ymax),(255,0,0),3)
        #cv2.imshow('out',image)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
        #cv2.rectangle(image,(region[0],region[1]),(region[1],region[3]),(255,0,0),15)
        #print(region)
        #plt.imshow(image)
        #plt.show()
'''

        box2 = [xmin, ymin, xmax, ymax]
        iou_ = iou(box1, box2)
        ious.append(iou_)
        if DEBUG:
            if iou_<0.7:
                print("IoU for {} is {}".format(image_name, iou_))
            
                cv2.rectangle(image_, (int(xmin/IMAGE_SIZE*width), int(ymin/IMAGE_SIZE*height)), (int(xmax/IMAGE_SIZE*width), int(ymax/IMAGE_SIZE*height)), (0, 0, 255), 1)
                cv2.rectangle(image_, (int(box1[0]/IMAGE_SIZE*width), int(box1[1]/IMAGE_SIZE*height)), (int(box1[2]/IMAGE_SIZE*width), int(box1[3]/IMAGE_SIZE*height)), (0, 255, 0), 1)
            
                cv2.imshow("image", image_)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
        np.set_printoptions(suppress=True)
        print("\nAvg IoU: {}".format(np.mean(ious)))
        print("Highest IoU: {}".format(np.max(ious)))
        print("Lowest IoU: {}".format(np.min(ious)))

'''
    

if __name__ == "__main__":
    main()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

100%|██████████| 12815/12815 [17:24<00:00, 12.10it/s]


In [0]:
new_df=pd.DataFrame()

In [0]:
new_df['image_name']=lab['image_name']
new_df['x1']=xmin
new_df['x2']=xmax
new_df['y1']=ymin
new_df['y2']=ymax

In [24]:
new_df.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,178.974899,459.781708,92.206085,400.934357
1,1473231475010DeeplearnS11276.png,58.381042,602.301514,165.285110,357.662994
2,JPEG_20161205_135307_1000155917326.png,131.148865,491.989319,68.087227,406.303558
3,JPEG_20160711_123440_1000518778437.png,161.082245,470.159851,129.120026,402.302704
4,JPEG_20160803_115329_100034020722.png,98.714394,527.139709,77.477692,407.481232


In [0]:
new_df.to_csv('Resnet_15E_14.csv', index=False)

In [0]:
from google.colab import files
files.download('Resnet_15E_14.csv')

In [0]:
!ls

adc.json			     __pycache__
FlipkartGridStage2DataSetImages.zip  Resnet_5E.csv
gdrive				     sample_data
input				     tb1
Mobile-net.h5			     train.py
ngrok				     weights
ngrok-stable-linux-amd64.zip	     weights_Res.zip


In [0]:
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
filename = "Mobile-net.h5"
def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ""

zip_file = filename + extension_zip

# !rm -rf $zip_file
#!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)

File ID: 1UKFEDHEWPPm31wSC1FxVdWR-4FBzSgHr


{'id': '1UKFEDHEWPPm31wSC1FxVdWR-4FBzSgHr'}

In [0]:
!ls tb1

events.out.tfevents.1549729540.aef18aa08f8d
